In [9]:
import sys, os
pbin_dir = os.path.dirname(sys.executable)
%env PBIN_DIR={pbin_dir}

env: PBIN_DIR=/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin


In [1]:
#!${PBIN_DIR}/pip install gensim
import json
import pydash
from torchtext.data import get_tokenizer
from gensim.models import word2vec
tokenizer = get_tokenizer("basic_english")
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
    jsond = json.loads(f.read())
sentences = []
for author in jsond:
    hists = jsond[author]
    for hist in hists:
        for key in hist:
            if key.startswith('context/'):
                sentences.append(hist[key])
            sentences.append(hist['Context'])
            sentences.append(hist['Response'])
sentences = pydash.uniq_by(sentences)
wordss = [tokenizer(sentence) for sentence in sentences]
#padlen = 50
#for i, words in enumerate(wordss):
#    wordss[i] = ['<PAD>'] * (padlen - len(words)) + words
print(wordss[0])


/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['it', "'", 's', 'impossible', 'at', 'the', 'moment', ',', 'but', 'can', 'you', 'imagine', 'him', 'on', 'the', 'eagles', 'with', 'their', 'd-line', '?']


In [2]:
import multiprocessing
#https://radimrehurek.com/gensim/models/word2vec.html#
#https://rare-technologies.com/word2vec-tutorial/
model = word2vec.Word2Vec(wordss,vector_size=200,workers=multiprocessing.cpu_count())


In [3]:
wv = model.wv
vocab = {'<PAD>':0}
#PADは本家でも0になってる
embeddings = [[0.0]*200]
print(len(wv))
for i in range(len(wv)):
    vocab[wv.index_to_key[i]] = i+1
    #normalizeして出力する方法もあるがcikmのembeddingみるとしてなさそう
    embeddings.append(wv[i])
import pickle
with open("vocab_and_embeddings_rednfl.pkl", mode="wb") as f:
    pickle.dump([vocab,embeddings], f)


52356


In [32]:
!cp dataset

reddit		reddit2		    reddit_orig		      weibo
reddit.tar.bz2	reddit_emb.tar.bz2  vocab_and_embeddings.pkl
